In [16]:
from psycopg_pool import ConnectionPool
import os

In [17]:
pool = ConnectionPool(
    os.getenv("SUPABASE_CONECTION_URL")
)

In [18]:
with pool.connection() as conn:
    with conn.cursor() as cur:
        cur.execute("SELECT * FROM financelive.users")
        rows = cur.fetchall()
        for row in rows:
            print(row)

(1, 'testuser0@test.com', 'testuser', 'lost', datetime.date(2003, 1, 29), 'test', datetime.datetime(2025, 3, 30, 1, 38, 36, 467736))


In [19]:
def use_connection(sql: str, params=None):
    if params is None:
        print("no params")
    with pool.connection() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, params)
            rows = cur.fetchall()
            for row in rows:
                print(row)

In [20]:
## PROBAR LAS CONSULTAS
sql = """SELECT * FROM financelive.users WHERE email = %s"""
params = ("testuser0@test.com", )
use_connection(sql, params)

(1, 'testuser0@test.com', 'testuser', 'lost', datetime.date(2003, 1, 29), 'test', datetime.datetime(2025, 3, 30, 1, 38, 36, 467736))


In [24]:
sql = """
SELECT SUM(i.amount) AS total_ingresos_mes
FROM financelive.incomes i
JOIN financelive.users u ON i.user_id = u.id
WHERE u.email = %s
  AND EXTRACT(MONTH FROM i.date_created) = %s
  AND EXTRACT(YEAR FROM i.date_created) = %s
"""


In [25]:
from datetime import date
today = date.today()

params = ("testuser0@test.com", today.month, today.year)

use_connection(sql, params)


(None,)


In [37]:
sql = """
SELECT TO_CHAR(i.date_created, 'YYYY-MM') AS mes,
       COALESCE(SUM(i.amount), 0) AS total
FROM financelive.incomes i
JOIN financelive.users u ON i.user_id = u.id
WHERE u.email = %s
GROUP BY mes
ORDER BY mes DESC
LIMIT 6
"""

params = ("testuser0@test.com", )
use_connection(sql, params)


In [38]:
sql = """
SELECT u.email, COALESCE(SUM(i.amount), 0) AS total_ingresos
FROM financelive.incomes i
JOIN financelive.users u ON i.user_id = u.id
GROUP BY u.email
"""

use_connection(sql)


no params


In [39]:
sql = """
SELECT u.email, COUNT(i.id) AS cantidad_movimientos
FROM financelive.incomes i
JOIN financelive.users u ON i.user_id = u.id
GROUP BY u.email
ORDER BY cantidad_movimientos DESC
"""

use_connection(sql)


no params


In [40]:
sql = """
SELECT u.email, SUM(i.amount) AS total
FROM financelive.incomes i
JOIN financelive.users u ON i.user_id = u.id
GROUP BY u.email
ORDER BY total DESC
LIMIT 5
"""

use_connection(sql)


no params


In [41]:
sql = """
SELECT i.amount, i.description_, i.date_created
FROM financelive.incomes i
JOIN financelive.users u ON i.user_id = u.id
WHERE u.email = %s
ORDER BY i.date_created DESC
LIMIT 10
"""
params = ("testuser0@test.com", )
use_connection(sql, params)


In [42]:
params = ("testuser0@test.com", )

In [43]:
sql = """
SELECT u.email, COALESCE(SUM(i.amount), 0) AS total_ingresos
FROM financelive.incomes i
JOIN financelive.users u ON i.user_id = u.id
WHERE u.email = %s
GROUP BY u.email
"""
params = ("testuser0@test.com", )
use_connection(sql, params)


In [44]:
sql = """
SELECT TO_CHAR(i.date_created, 'YYYY-MM') AS mes,
       COALESCE(SUM(i.amount), 0) AS total
FROM financelive.incomes i
JOIN financelive.users u ON i.user_id = u.id
WHERE u.email = %s
GROUP BY mes
ORDER BY mes DESC
LIMIT 6
"""
params = ("testuser0@test.com", )
use_connection(sql, params)


In [45]:
sql = """
SELECT i.amount, i.description_, i.date_created
FROM financelive.incomes i
JOIN financelive.users u ON i.user_id = u.id
WHERE u.email = %s
ORDER BY i.date_created DESC
LIMIT 10
"""
params = ("testuser0@test.com", )
use_connection(sql, params)


In [46]:
from datetime import date
today = date.today()

sql = """
SELECT COALESCE(SUM(i.amount), 0) AS total_mes
FROM financelive.incomes i
JOIN financelive.users u ON i.user_id = u.id
WHERE u.email = %s
  AND EXTRACT(MONTH FROM i.date_created) = %s
  AND EXTRACT(YEAR FROM i.date_created) = %s
"""
params = ("testuser0@test.com", today.month, today.year)
use_connection(sql, params)


(Decimal('0'),)


In [47]:
sql = """
SELECT *
FROM financelive.incomes
LIMIT 5
"""
use_connection(sql)

no params


In [52]:
from datetime import date

def ingresos_total_mes(email: str):
    today = date.today()
    sql = """
    SELECT COALESCE(SUM(i.amount), 0) AS total_mes
    FROM financelive.incomes i
    JOIN financelive.users u ON i.user_id = u.id
    WHERE u.email = %s
      AND EXTRACT(MONTH FROM i.date_created) = %s
      AND EXTRACT(YEAR FROM i.date_created) = %s
    """
    params = (email, today.month, today.year)
    resultado = use_connection(sql, params)
    if not resultado or resultado == [(None,)]:
        print("Sin ingresos registrados este mes.")
    else:
        print(f"Ingreso total del mes: {resultado[0][0]}")


In [53]:
ingresos_total_mes("testuser0@test.com")

(Decimal('0'),)
Sin ingresos registrados este mes.


In [54]:
def ingresos_historicos(email: str):
    sql = """
    SELECT COALESCE(SUM(i.amount), 0) AS total_historico
    FROM financelive.incomes i
    JOIN financelive.users u ON i.user_id = u.id
    WHERE u.email = %s
    """
    params = (email,)
    resultado = use_connection(sql, params)
    if not resultado or resultado == [(None,)]:
        print("Sin ingresos históricos registrados.")
    else:
        print(f"Ingreso total histórico: {resultado[0][0]}")


In [55]:
ingresos_historicos("testuser0@test.com")

(Decimal('0'),)
Sin ingresos históricos registrados.


In [56]:
def ultimos_ingresos(email: str):
    sql = """
    SELECT i.amount, i.description_, i.date_created
    FROM financelive.incomes i
    JOIN financelive.users u ON i.user_id = u.id
    WHERE u.email = %s
    ORDER BY i.date_created DESC
    LIMIT 10
    """
    params = (email,)
    resultado = use_connection(sql, params)
    if not resultado:
        print("Sin ingresos recientes.")
    else:
        for r in resultado:
            print(r)


In [57]:
ultimos_ingresos("testuser0@test.com")

Sin ingresos recientes.


In [58]:
def ingresos_por_mes(email: str):
    sql = """
    SELECT TO_CHAR(i.date_created, 'YYYY-MM') AS mes,
           COALESCE(SUM(i.amount), 0) AS total
    FROM financelive.incomes i
    JOIN financelive.users u ON i.user_id = u.id
    WHERE u.email = %s
    GROUP BY mes
    ORDER BY mes DESC
    LIMIT 6
    """
    params = (email,)
    resultado = use_connection(sql, params)
    if not resultado:
        print("Sin ingresos mensuales registrados.")
    else:
        for r in resultado:
            print(r)


In [59]:
ingresos_por_mes("testuser0@test.com")

Sin ingresos mensuales registrados.


In [60]:
from datetime import date

def saldo_neto_mensual(email: str):
    today = date.today()
    
    sql = """
    WITH ingresos AS (
      SELECT SUM(i.amount) AS total
      FROM financelive.incomes i
      JOIN financelive.users u ON i.user_id = u.id
      WHERE u.email = %s
        AND EXTRACT(MONTH FROM i.date_created) = %s
        AND EXTRACT(YEAR FROM i.date_created) = %s
    ),
    egresos AS (
      SELECT
        COALESCE((
          SELECT SUM(d.amount)
          FROM financelive.debts d
          JOIN financelive.users u ON d.user_id = u.id
          WHERE u.email = %s
            AND EXTRACT(MONTH FROM d.date_created) = %s
            AND EXTRACT(YEAR FROM d.date_created) = %s
        ), 0)
      +
        COALESCE((
          SELECT SUM(s.amount)
          FROM financelive.spendings s
          JOIN financelive.users u ON s.user_id = u.id
          WHERE u.email = %s
            AND EXTRACT(MONTH FROM s.date_created) = %s
            AND EXTRACT(YEAR FROM s.date_created) = %s
        ), 0)
      +
        COALESCE((
          SELECT SUM(inv.amount)
          FROM financelive.investments inv
          JOIN financelive.users u ON inv.user_id = u.id
          WHERE u.email = %s
            AND EXTRACT(MONTH FROM inv.date_created) = %s
            AND EXTRACT(YEAR FROM inv.date_created) = %s
        ), 0) AS total
    )
    SELECT
      COALESCE(ingresos.total, 0) AS ingresos_mes,
      COALESCE(egresos.total, 0) AS egresos_mes,
      COALESCE(ingresos.total, 0) - COALESCE(egresos.total, 0) AS saldo_neto
    FROM ingresos, egresos
    """
    
    params = (
        email, today.month, today.year,  # ingresos
        email, today.month, today.year,  # debts
        email, today.month, today.year,  # spendings
        email, today.month, today.year   # investments
    )
    
    resultado = use_connection(sql, params)
    
    if not resultado or resultado == [(None, None, None)]:
        print("No hay datos registrados para este mes.")
    else:
        ingresos, egresos, saldo = resultado[0]
        print(f"Ingresos del mes: {ingresos}")
        print(f"Gastos del mes: {egresos}")
        print(f"Saldo neto: {saldo}")


In [61]:
saldo_neto_mensual("testuser0@test.com")

(Decimal('0'), Decimal('1010000.00'), Decimal('-1010000.00'))
No hay datos registrados para este mes.


In [63]:
sql = """
INSERT INTO financelive.incomes (user_id, amount, description_, date_created)
VALUES 
  (1, 500.00, 'Pago freelance', NOW()),
  (1, 750.00, 'Reembolso', NOW())
"""

# Ejecutar sin esperar resultados
with pool.connection() as conn:
    with conn.cursor() as cur:
        cur.execute(sql)
        conn.commit()
        print("Ingresos insertados.")

Ingresos insertados.


In [64]:
saldo_neto_mensual("testuser0@test.com")

(Decimal('1250.00'), Decimal('1010000.00'), Decimal('-1008750.00'))
No hay datos registrados para este mes.


In [77]:
sql = """
INSERT INTO financelive.investments 
(user_id, amount, description_, status_, date_created)
VALUES 
  (1, 400.00, 'ETF mensual', TRUE, NOW()),
  (1, 150.00, 'Crypto', FALSE, NOW())
"""

with pool.connection() as conn:
    with conn.cursor() as cur:
        cur.execute(sql)
        conn.commit()
        print("Inversiones insertadas.")

Inversiones insertadas.


In [78]:
sql = """
INSERT INTO financelive.spendings (user_id, amount, description_, date_created)
VALUES 
  (1, 220.00, 'Comida', NOW()),
  (1, 80.00, 'Entretenimiento', NOW())
"""

with pool.connection() as conn:
    with conn.cursor() as cur:
        cur.execute(sql)
        conn.commit()
        print("Gastos insertados.")


Gastos insertados.


In [79]:
sql = """
INSERT INTO financelive.debts 
(user_id, amount, description_, interest_rate, total_payment, total_installments, date_created)
VALUES 
  (1, 300.00, 'Pago tarjeta', 2.5, 330.00, 12, NOW()),
  (1, 150.00, 'Crédito personal', 1.8, 160.00, 6, NOW())
"""

In [80]:
saldo_neto_mensual("testuser0@test.com")

(Decimal('1250.00'), Decimal('1011150.00'), Decimal('-1009900.00'))
No hay datos registrados para este mes.


In [82]:
sql = "SELECT id, email FROM financelive.users"
use_connection(sql)

no params
(1, 'testuser0@test.com')


In [83]:
saldo_neto_mensual("testuser0@test.com")


(Decimal('1250.00'), Decimal('1011150.00'), Decimal('-1009900.00'))
No hay datos registrados para este mes.


In [95]:
from datetime import date

def saldo_neto_mensual(email: str):
    today = date.today()

    sql = """
    SELECT
        COALESCE((
            SELECT SUM(i.amount)
            FROM financelive.incomes i
            JOIN financelive.users u ON i.user_id = u.id
            WHERE u.email = %s
            AND EXTRACT(MONTH FROM i.date_created) = %s
            AND EXTRACT(YEAR FROM i.date_created) = %s
        ), 0) AS total_ingresos,

        COALESCE((
            SELECT SUM(d.amount)
            FROM financelive.debts d
            JOIN financelive.users u ON d.user_id = u.id
            WHERE u.email = %s
            AND EXTRACT(MONTH FROM d.date_created) = %s
            AND EXTRACT(YEAR FROM d.date_created) = %s
        ), 0) +
        COALESCE((
            SELECT SUM(s.amount)
            FROM financelive.spendings s
            JOIN financelive.users u ON s.user_id = u.id
            WHERE u.email = %s
            AND EXTRACT(MONTH FROM s.date_created) = %s
            AND EXTRACT(YEAR FROM s.date_created) = %s
        ), 0) +
        COALESCE((
            SELECT SUM(inv.amount)
            FROM financelive.investments inv
            JOIN financelive.users u ON inv.user_id = u.id
            WHERE u.email = %s
            AND EXTRACT(MONTH FROM inv.date_created) = %s
            AND EXTRACT(YEAR FROM inv.date_created) = %s
        ), 0) AS total_gastos
    """

    params = (
        email, today.month, today.year,
        email, today.month, today.year,
        email, today.month, today.year,
        email, today.month, today.year,
    )

    resultado = use_connection(sql, params)

    if resultado and resultado[0] and any(v is not None for v in resultado[0]):
        ingresos, gastos = resultado[0]
        saldo = ingresos - gastos
        print(f"Ingresos: ${ingresos}")
        print(f"Gastos: ${gastos}")
        print(f"Saldo neto: ${saldo}")
    else:
        print("No hay datos registrados para este mes.")



In [96]:
saldo_neto_mensual("testuser0@test.com")

(Decimal('1250.00'), Decimal('1011150.00'))
No hay datos registrados para este mes.
